In [1]:
import numpy as np

import tensorflow as tf

from PIL import Image as Img
import os

import mtcnn

from os import listdir, environ
from os.path import isfile, join
import time

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

my_image_path = "../../faces/"
dataset_image_path = "../../../Dataset/data/lfw/"
database = r"../../../Dataset/database/SmartFinder.db"

## Load Model

In [2]:
tf_lite_model = tf.lite.Interpreter(model_path='../../tflite/facenet.tflite')
tf_lite_model.allocate_tensors()

input_details = tf_lite_model.get_input_details()
output_details = tf_lite_model.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 160 160   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [  1 512]
type: <class 'numpy.float32'>


In [3]:
def get_image_embedding(image_path, tf_lite_model):
    # Open image
    image = Img.open(image_path).convert('RGB')
    pixels = np.asarray(image)
    # detect faces in the image
    detector = mtcnn.MTCNN()
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Img.fromarray(face)
    image = image.resize((160, 160))
    face_array = np.asarray(image)
    face_array_norm = face_array /255
    
    input_details = tf_lite_model.get_input_details()
    output_details = tf_lite_model.get_output_details()

    tf_lite_model.set_tensor(input_details[0]['index'], face_array_norm.reshape(1,160,160,3).astype(np.float32))

    tf_lite_model.invoke()

    return tf_lite_model.get_tensor(output_details[0]['index'])

## Load Database

In [4]:
import time
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime, timedelta
#from flask_cors import CORS

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///../../../Dataset/database/SmartFinder1000.db' #'sqlite:///test.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

#cors = CORS(app, resources={r"/*": {"origins": "*"}})

      #############
########  Utils  ########
      #############

import numpy as np
def get_distance(embedding_1, embedding_2):
    return np.linalg.norm(np.array(eval(embedding_1)) - np.array(eval(embedding_2)))

def is_new_match(json_match, new_name):
    for match in json_match:
        if match['person_name'] == new_name:
            return False
    return True

In [5]:
      #############
########  Models ########
      #############

class Person(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    name = db.Column(db.String(200), nullable=False)
    birthday = db.Column(db.DateTime, nullable=False)
    status = db.Column(db.String(200), nullable=False)
    created_at = db.Column(db.DateTime, nullable=True)
    updated_at = db.Column(db.DateTime, nullable=True)
    images = db.relationship('Image', uselist=True, backref='person')
    matches = db.relationship('Match', uselist=True, backref='person')

class Image(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    person_id = db.Column(db.Integer, db.ForeignKey('person.id'), nullable=True)
    path = db.Column(db.String(200), nullable=False)
    embedding = db.Column(db.String(500), nullable=False)
    created_at = db.Column(db.DateTime, nullable=True)
    matches = db.relationship('Match', uselist=True, backref='image')

class Request(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    lat = db.Column(db.Integer, nullable=False)
    long = db.Column(db.Integer, nullable=False)
    embedding = db.Column(db.String(500), nullable=False)
    created_at = db.Column(db.DateTime, nullable=True)
    matches = db.relationship('Match', uselist=True, backref='request')

class Match(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    request_id = db.Column(db.Integer, db.ForeignKey('request.id'), nullable=True)
    image_id = db.Column(db.Integer, db.ForeignKey('image.id'), nullable=True)
    person_id = db.Column(db.Integer, db.ForeignKey('person.id'), nullable=True)
    distance = db.Column(db.Float, nullable=True)

In [6]:
image_list = Image.query.all()

db_embeddings = np.array([eval(image.embedding) for image in image_list])

In [7]:
person_list = Person.query.all()

person_ids = np.array([person.id for person in person_list])

In [8]:
print("Pessoas na base: ", len(person_ids))

Pessoas na base:  1000


## Load new embeddings

In [9]:
class NewPerson():
    def __init__(self, name, embedding, _id):
        self.name = name
        self.embedding = embedding
        self.id = _id

In [11]:
start_time = time.time()

new_person_list = []

_id = 1
for person_name in sorted(listdir(dataset_image_path))[0:6]:

    for image_path in sorted(listdir(dataset_image_path+person_name))[3:4]:
        embedding = get_image_embedding(dataset_image_path+person_name+'/'+image_path, tf_lite_model)
        person = NewPerson(person_name, embedding, _id)
        new_person_list.append(person)

        print("Person {}: {} --- {:0.2f} minutes ---".format(_id, person_name, (time.time() - start_time)/60))
    _id += 1
    
print("\nTotal time: {:0.2f} minutes".format((time.time() - start_time)/60))

new_embeddings = np.array([person.embedding for person in new_person_list])
new_person_ids = np.array([person.id for person in new_person_list])

print("{} pessoas novas".format(new_person_ids.shape[0]))

Person 6: Aaron_Peirsol --- 0.07 minutes ---

Total time: 0.07 minutes
1 pessoas novas


## Get distances

In [12]:
match_distance = 0.8

In [13]:
distances = np.linalg.norm(new_embeddings[:, None, :] - db_embeddings[None, :, :], axis=-1)

In [14]:
distances.shape

(1, 1295, 1)

In [14]:
#sorted(distances[43])

In [15]:
verdadeiro_positivo = 0
verdadeiro_negativo = 0
falso_positivo = 0
falso_negativo = 0

for i in range(0, len(distances)):
    if new_person_ids[i] in person_ids:
        # Está no banco de dados e ...
        if new_person_ids[i] in [image_list[index].person_id for index in (np.where(distances[i]<match_distance)[0])]:
            # encontrou
            verdadeiro_positivo += 1
        else:
            # não encontrou
            verdadeiro_negativo += 1
            
    else:
        # Não está no banco de dados e ...
        if len(np.where(distances[i]<match_distance)[0]) > 0:
            # encontrou alguém
            falso_positivo += 1
        else:
            # não encontrou
            falso_negativo += 1
            
print("verdadeiro_positivo: ", verdadeiro_positivo)
print("verdadeiro_negativo: ", verdadeiro_negativo)
print("falso_positivo: ", falso_positivo)
print("falso_negativo: ", falso_negativo)

verdadeiro_positivo:  83
verdadeiro_negativo:  26
falso_positivo:  7
falso_negativo:  91
